In [1]:
import json
import re
import numpy as np

In [2]:
def retrieve_data():
    
    en_prop_key = {
        'budget':'लागत',
        'narr locations' : 'कथा स्थल',
        'production company' : 'उत्पादन कंपनी',
        'artist' : 'कार्यकारी निर्माता',
        'ICAA rating' : 'आईसीएए रेटिंग',
        'series' : 'श्रृंखला का हिस्सा',
        'NMHH rating' : 'एनएमएचएच फिल्म रेटिंग' ,
        'nominated' : 'के लिए मनोनीत',
        'sequel' : 'बाद' ,
        'MPAA rating' : 'एमपीएए फिल्म रेटिंग',
        'date' : 'प्रकाशन तिथि',
        'release year' : 'प्रकाशन तिथि',
        'awards won' : 'पुरस्कार प्राप्त',
        'cast' : 'कास्ट मेंबर',
        'producer' : 'निर्माता' ,
        'website' : 'आधिकारिक वेबसाइट' ,
        'distributor' : 'द्वारा वितरित' ,
        'based on' : 'आधार पर',
        'director' : 'निदेशक',
        'country' : 'मूल देश',
        'editor' : 'फिल्म संपादक',
        'production designer' : 'प्रोडक्शन डिजाइनर',
        'singers' : 'संगीतकार',
        'topic' : 'मुख्य विषय',
        'fil locations' : 'फिल्मांकन स्थान',
        'music director' : 'कलाकार',
        'dir photo' : 'फोटोग्राफी के निदेशक',
        'FSK rating' : 'एफएसके फिल्म रेटिंग',
        'duration' : 'पहलू अनुपात',
        'colour': 'रंग',
        'box office' : 'बॉक्स ऑफिस',
        'costume designer' : 'कॉस्ट्यूम डिजाइनर',
        'lang' : 'फिल्म या टीवी शो की मूल भाषा',
        'screenwriter' : 'पटकथा लेखक' ,
        'genre' : 'शैली' 
    }

    
    
    sentence_templates = []
    with open('../Data/Films/Films_Generated_Wikipages/template_sents.json') as f:
        sentence_templates = json.load(f)

    hi_labelled_films = {}
    with open('../Data/Films/Films_Hindi_Wikidata_Hi_Labelled/films_hi_labelled.json') as f:
        hi_labelled_films = json.load(f)
    
    film_ids = []
    with open('../Data/Films/WikiBio_Styled_Formatted_Data/train/train.id') as f:
        film_ids = f.readlines()
    film_ids = [x.strip() for x in film_ids]
    
    film_titles = []
    with open('../Data/Films/WikiBio_Styled_Formatted_Data/train/train.title') as f:
        film_titles = f.readlines()
    film_titles = [x.strip() for x in film_titles]
    
    film_urls = []
    with open('../Data/Films/WikiBio_Styled_Formatted_Data/train/train.url') as f:
        film_urls = f.readlines()
    film_urls = [x.strip() for x in film_urls]
    
    film_details = {}
    
    for i in range(0,len(film_ids)):
        film_details[film_ids[i]] = {'url':film_urls[i], 'title': film_titles[i]}
        
    return film_details,sentence_templates, hi_labelled_films, en_prop_key

In [3]:
def get_first_sentences_templates(sentence_templates):
    first_sentences = []
    for sentence in sentence_templates:
        if '{{film}}' in sentence.split():
            first_sentences.append(sentence)
    
    return first_sentences

In [4]:
def retrieve_template_tags(sentence_templates):
    template_tags = {}
    for sentence_template in sentence_templates:
        template_tags[sentence_template] = re.findall(r'\{\{(.*?)\}\}', sentence_template)
    
    return template_tags

In [5]:
def retrieve_sent_tags(sentence):
    return re.findall(r'\{\{(.*?)\}\}', sentence)

In [6]:
def retrieve_first_sentence_template(template_tags, article_props, en_prop_key):
    sentence_templates = list(template_tags.keys())
    first_sentences = get_first_sentences_templates(sentence_templates)
    first_sentences_tags = retrieve_template_tags(first_sentences)
    
    for sent in first_sentences_tags.keys():
        tags = first_sentences_tags[sent]
        tags.remove('film')
        first_sentences_tags[sent] = tags
    
    matching_template = ""
    max_template_size = 0
    
    for first_sent in first_sentences_tags.keys():
        tags  = first_sentences_tags[first_sent]
        hi_tags = [en_prop_key[tag] for tag in tags]
        
        if set(hi_tags) <= set(article_props) and len(hi_tags) > max_template_size:
            max_template_size = len(hi_tags)
            matching_template = first_sent
            
    return matching_template
    

In [7]:
def retrieve_remaining_sentence_template(remaining_template_tags, remaining_article_props, en_prop_key):    
    matching_template = ""
    max_template_size = 0
    
    for remaining_sent in remaining_template_tags.keys():
        tags  = remaining_template_tags[remaining_sent]
        hi_tags = [en_prop_key[tag] for tag in tags]
        
        if set(hi_tags) <= set(remaining_article_props) and len(hi_tags) > max_template_size:
            max_template_size = len(hi_tags)
            matching_template = remaining_sent
            
    return matching_template

In [8]:
def update_remaining_props(template_tags, article_props, article, en_prop_key):
    article_tags = retrieve_sent_tags(article)
    new_template_tags = {}
    
    for template_sent in template_tags.keys():
        flag = 1
        tags = template_tags[template_sent]
        for tag in article_tags:
            if tag in tags:
                flag = 0
                break
        if flag == 1:
            new_template_tags[template_sent] = template_tags[template_sent]
    
    article_tags.remove('film')
    used_props = [en_prop_key[tag] for tag in article_tags]
    
    new_article_props = []
    for prop in article_props:
        if prop not in used_props:
            new_article_props.append(prop)
    
    return new_template_tags, new_article_props
    

In [9]:
def create_wikipage(film_id, film_details, sentence_templates, hi_labelled_films, en_prop_key):
    template_tags = retrieve_template_tags(sentence_templates)
    total_props = np.array(list(en_prop_key.values()))
    total_props = list(np.unique(total_props))
    
    film = hi_labelled_films[film_id]
    film_wiki_props = {}
    
    for prop in total_props:
        if prop in film.keys():
            film_wiki_props[prop] = film[prop]
    
    article_props = set(list(film_wiki_props.keys()))
    non_article_props = set(['पहलू अनुपात', 'आईसीएए रेटिंग', 'एफएसके फिल्म रेटिंग', 'आधिकारिक वेबसाइट', 'उत्पादन कंपनी', 'एमपीएए फिल्म रेटिंग', 'एनएमएचएच फिल्म रेटिंग'])
    article_props = article_props.difference(non_article_props)
    article_props = list(article_props)
    
    article = retrieve_first_sentence_template(template_tags, article_props , en_prop_key)
    if article == "":
        article = "{{film}} एक फिल्म है।"
        
    while article_props:
        template_tags, article_props = update_remaining_props(template_tags, article_props, article, en_prop_key)
        article = article + retrieve_remaining_sentence_template(template_tags, article_props, en_prop_key)
    
    
    return article

In [10]:
film_details,sentence_templates, hi_labelled_films, en_prop_key = retrieve_data()
print(create_wikipage('Q182218', film_details, sentence_templates, hi_labelled_films, en_prop_key))

{{film}} {{release year}} में बनी {{genre}} शैली की फ़िल्म है जिसका निर्माण {{artist}} ने किया है व वितरण {{distributor}} द्वारा किया गया है। {{producer}} द्वारा निर्मित इस फिल्म का लेखन {{screenwriter}} ने किया और निर्देशन {{director}} ने संम्भाला।यह फिल्म {{based on}} पर आधारित {{topic}} के विषय पे बनी फिल्म है । इस फिल्म की घटनाए {{narr locations}} पर हुई गयी थी लेकिन इनकी शूटिंग {{fil locations}} पर की गयी थी। फिल्म के सम्पादक {{editor}} थे और इनके साथ फोटोग्राफी निर्देशन {{dir photo}} ने संभाली ।यह मौलिक रूप से {{lang}} भाषा में शूट हुई एक {{country}} फिल्म है।इस फिल्म ने कई पुरस्कार मनोनीत  किया गया जैसे {{nominated}} और इन में से कई पुरस्कार जैसे {{awards won}} जीते भी।फिल्म की लागत {{budget}} थी और उसकी तुलना में फिल्म ने {{box office}} कमाए।इस फिल्म में {{singers}} ने गायन किया है।यह एक {{colour}} फिल्म है। यह फिल्म {{series}} श्रृंखला का हिस्सा है। फिल्म की  श्रृंखला में अगली फिल्म {{sequel}} है।{{cast}} ने इस फिल्म में भूमिका निभाई ।फिल्म के लिए पोशाक रचना {{costume designer